# Setup and imports

In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
import matplotlib.pyplot as plt
import seaborn           as sb
import pandas            as pd

sb.set()
sb.set_style("white")

In [ ]:
import json
import networkx as nx
import graph_tool as gt
import numpy as np

from graph_tool.inference import minimize_nested_blockmodel_dl
from infomap import Infomap

# Function definitions

In [ ]:
# Loads a network from the given file and returns it as a networkx network.
def loadNetwork(filename):
    with open(filename) as fh:
        data = json.loads(fh.read())
        
    G = nx.DiGraph()

    for n in data["nodes"]:
        G.add_node(int(n["data"]["id"]))

    for k in data["edges"]:
        e = data["edges"][k]
        if "source" in e["data"] and "target" in e["data"]:
            u,v,w = e["data"]["source"], e["data"]["target"], e["style"]["width"]
            G.add_edge(int(u), int(v), weight = float(w))
    
    return G

In [ ]:
# Saves the given network in net format for Infomap
def toNet(G, filename):
    with open(filename, "w") as fh:
        fh.write("*Vertices {:}\n".format(len(G.nodes)))
        for n in G.nodes:
            fh.write("{:} \"{:}\"\n".format(n, n))
        fh.write("*Edges {:}\n".format(len(G.edges)))
        for u,v in G.edges:
            fh.write("{:} {:} {:}\n".format(u, v, G[u][v]["weight"]))

In [ ]:
# Plots a networkx network
def plotNetwork(G):
    weights = [G[u][v]["weight"] for u,v in G.edges]
    labels  = dict([(n, str(n)) for n in G.nodes])
    pos     = nx.kamada_kawai_layout(G)

    nx.draw(G, pos, width = weights)
    nx.draw_networkx_labels(G, pos, labels, font_size=8)
    plt.show()

In [ ]:
# Takes a networkx network and converts it to a graph-tool network
def toGraphTool(G):
    H = gt.Graph(directed = False)
    node_labels  = H.new_vertex_property("string")
    edge_weights = H.new_edge_property("double")
    H.edge_properties["weight"] = edge_weights
    num_vertices = len(G.nodes)
    for n in G.nodes:
        node_labels[n] = str(n)
    for u,v in G.edges:
        e = H.add_edge(u, v)
        edge_weights[e] = G[u][v]["weight"]
    H.vertex_properties["label"] = node_labels
    return H

# Load a network and plot it

In [ ]:
G = loadNetwork("data/attack5.json")
toNet(G, "data/attack5.net")

In [ ]:
plotNetwork(G)

# Convert it and infer nested block structure with degree correction

In [ ]:
H = toGraphTool(G)

In [ ]:
s = minimize_nested_blockmodel_dl(H, deg_corr=True)

In [ ]:
s.draw(vertex_text=H.vertex_properties['label'], output_size=(1024,1024))

# Find communities with Infomap

In [ ]:
bestModules = dict()
bestCodelength = inf

for i in range(1000):
    im = Infomap("--silent")
    for u,v in G.edges:
        im.addLink(u,v,G[u][v]["weight"])
    for n in G.nodes:
        im.addNode(n)
    
    im.run()
    if im.getCodelength() < bestCodelength:
        bestCodelength = im.getCodelength()
        bestModules = im.getMultilevelModules()